# Descriptive analysis for the manuscript

Summarize geotagged tweets of the multiple regions used for the experiment and the application.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import yaml
import scipy.stats as stats

def load_region_tweets(region=None):
    df = pd.read_csv(f'../../dbs/{region}/geotweets.csv')
    df['day'] = df['createdat'].apply(lambda x: x.split(' ')[0])
    df['createdat'] = pd.to_datetime(df['createdat'], infer_datetime_format=True)
    t_max, t_min = df.createdat.max(), df.createdat.min()
    time_span = f'{t_min} - {t_max}'
    num_users = len(df.userid.unique())
    num_geo = len(df)
    num_days = np.median(df.groupby(['userid'])['day'].nunique())
    num_geo_freq = np.median(df.groupby(['userid']).size() / df.groupby(['userid'])['day'].nunique())
    return region, time_span, num_users, num_geo, num_days, num_geo_freq

region_list = ['sweden', 'netherlands', 'saopaulo', 'australia', 'austria', 'barcelona',
               'capetown', 'cebu', 'egypt', 'guadalajara', 'jakarta',
               'johannesburg', 'kualalumpur', 'lagos', 'madrid', 'manila', 'mexicocity', 'moscow', 'nairobi',
               'rio', 'saudiarabia', 'stpertersburg', 'surabaya']

with open('../../lib/regions.yaml', encoding='utf8') as f:
    region_manager = yaml.load(f, Loader=yaml.FullLoader)

## 1 Summarize the geotagged tweets used as input to the model
Geotagged tweets: Time span, No. of Twitter users, No. of geotagged tweets,
Days covered/user, No. of geotagged tweets/day/user

In [7]:
df = pd.DataFrame([load_region_tweets(region=x) for x in region_list],
                  columns=('region', 'time_span', 'num_users', 'num_geo', 'num_days', 'num_geo_freq'))
df.loc[:, 'gdp_capita'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['gdp_capita'])
df.loc[:, 'country'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['country'])
df.loc[:, 'pop'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['pop'])
df.loc[:, 'time_span'] = df.loc[:, 'time_span'].apply(lambda x: ' - '.join([x_t.split(' ')[0] for x_t in x.split(' - ')]))
df.loc[:, 'region'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['name'])
df

region                time_span  num_users  num_geo  \
0                       Sweden  2010-09-15 - 2019-03-31       3961  1248158   
1              The Netherlands  2010-09-12 - 2019-04-22       5375  1479674   
2            São Paulo, Brazil  2010-09-15 - 2019-06-07      10943  3513796   
3                    Australia  2010-09-11 - 2019-05-09       3310   847668   
4                      Austria  2010-09-13 - 2019-09-23        729   168364   
5             Barcelona, Spain  2010-09-14 - 2019-06-24       1891   482357   
6      Cape Town, South Africa  2011-01-20 - 2019-06-05       1092   193993   
7            Cebu, Philippines  2010-09-16 - 2019-09-22       1486   400094   
8                        Egypt  2010-10-06 - 2019-09-26       1464   234322   
9          Guadalajara, Mexico  2011-02-10 - 2019-09-19        684   212681   
10          Jakarta, Indonesia  2010-09-11 - 2019-06-20      13088  4650699   
11  Johannesburg, South Africa  2010-10-08 - 2019-07-23       1268   214187   
12      Kuala Lumpur, Malaysia  2010-09-14 - 2019-09-18       4663  1543306   
13              Lagos, Nigeria  2011-01-10 - 2019-08-28        812    96943   
14               Madrid, Spain  2010-10-19 - 2019-07-19       3172   671234   
15         Manila, Philippines  2010-09-14 - 2019-08-25      11997  3482176   
16         Mexico City, Mexico  2010-09-14 - 2019-07-09      15615  4613752   
17              Moscow, Russia  2010-09-14 - 2019-08-02       4206  1628759   
18              Nairobi, Kenya  2011-07-15 - 2019-07-27        644   166404   
19      Rio de Janeiro, Brazil  2010-09-12 - 2019-05-23       6063  1593782   
20                Saudi Arabia  2010-10-09 - 2019-04-29       3117   626427   
21    Saint Petersburg, Russia  2010-09-26 - 2019-07-29       1386   508124   
22         Surabaya, Indonesia  2010-09-16 - 2019-05-01       2414   803844   

    num_days  num_geo_freq  gdp_capita             country    pop  
0      111.0      1.432292       54.61              Sweden  10.23  
1      100.0      1.402878       53.02     The Netherlands  17.28  
2       96.0      1.519231       27.13              Brazil  12.18  
3       84.0      1.457786       57.40           Australia  24.99  
4       88.0      1.357143       51.50             Austria   8.56  
5       95.0      1.371429       33.10               Spain   5.56  
6       62.0      1.517549        6.20        South Africa   0.43  
7       83.0      1.593162        3.10         Philippines   0.92  
8       47.0      1.642324        2.55               Egypt  98.42  
9      102.5      1.490107       17.88              Mexico   1.50  
10     117.0      1.718750       19.03           Indonesia   9.61  
11      52.0      1.731532       16.37        South Africa   0.96  
12     104.0      1.590461       11.14            Malaysia   1.81  
13      36.0      1.806401        2.22             Nigeria  21.00  
14      75.0      1.398097       48.77               Spain   6.64  
15      93.0      1.594937        9.60         Philippines   1.78  
16     104.0      1.500000       10.12              Mexico   8.56  
17     119.0      1.736842       22.06  Russian Federation  11.92  
18      34.0      3.165152        2.01               Kenya   4.40  
19      77.0      1.516393       11.02              Brazil   6.32  
20      65.0      1.515625       23.34        Saudi Arabia  33.70  
21     106.0      1.737898       12.40  Russian Federation   4.99  
22     110.0      1.636364       14.20           Indonesia   2.77

In [8]:
df.to_clipboard(index=False)

## 2 Merge ODMs for visualisation
This part applies to Sweden, The Netherlands, and Sao Paulo, Brazil.

Separate files will be deleted.

In [7]:
for region in ['sweden', 'netherlands', 'saopaulo']:
    df = pd.read_csv(f'../../dbs/{region}/odm_gt.csv')
    df_c = pd.read_csv(f'../../dbs/{region}/calibration_odm.csv')
    df_v = pd.read_csv(f'../../dbs/{region}/validation_odm.csv')
    df = pd.merge(df, df_c, on=['ozone', 'dzone'])
    df = df.rename(columns={'model': 'model_c'})
    df = pd.merge(df, df_v, on=['ozone', 'dzone'])
    df = df.rename(columns={'model': 'model_v'})
    df.loc[:, ['ozone', 'dzone', 'gt', 'model_c', 'model_v']].to_csv(f'../../dbs/{region}/odms.csv', index=False)

## 3 Quantify the od-pair similarity
This part applies to Sweden, The Netherlands, and Sao Paulo, Brazil.

The overall similarity.

In [6]:
for region in ['sweden', 'netherlands', 'saopaulo']:
    df = pd.read_csv(f'../../dbs/{region}/odms.csv')
    print(region, 'GT vs. calibrated model', stats.kendalltau(df.loc[:, 'gt'], df.loc[:, 'model_c']))
    print(region, 'GT vs. validated model', stats.kendalltau(df.loc[:, 'gt'], df.loc[:, 'model_v']))

sweden GT vs. calibrated model KendalltauResult(correlation=0.12707912414478398, pvalue=0.0)
sweden GT vs. validated model KendalltauResult(correlation=0.12001211410870506, pvalue=0.0)
netherlands GT vs. calibrated model KendalltauResult(correlation=0.20725705094829874, pvalue=0.0)
netherlands GT vs. validated model KendalltauResult(correlation=0.21088191229229744, pvalue=0.0)
saopaulo GT vs. calibrated model KendalltauResult(correlation=0.3129177858473001, pvalue=0.0)
saopaulo GT vs. validated model KendalltauResult(correlation=0.3081899966684245, pvalue=0.0)


## 4 Check the domestic PKT of The Netherlands